## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../"+"Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Ushuaia,AR,-54.8000,-68.3000,62.26,31,20,16.11,few clouds
1,1,Hobart,AU,-42.8794,147.3294,48.11,95,90,5.01,moderate rain
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,83,75,13.80,broken clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,60.91,51,75,16.11,broken clouds
4,4,Talnakh,RU,69.4865,88.3972,-5.64,96,100,9.28,overcast clouds


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,83,75,13.80,broken clouds
6,6,Matongo,TZ,-2.3833,34.0833,75.83,49,99,3.36,overcast clouds
13,13,San Andres,CO,12.5847,-81.7006,87.82,74,75,4.61,broken clouds
14,14,Vaini,TO,-21.2000,-175.2000,73.56,83,75,6.91,broken clouds
16,16,Uong Bi,VN,21.0333,106.7833,71.58,93,98,2.86,overcast clouds
17,17,Lodwar,KE,3.1191,35.5973,88.00,34,13,8.10,few clouds
18,18,Saint George,US,37.1041,-113.5841,73.87,43,1,1.01,clear sky
19,19,Rikitea,PF,-23.1203,-134.9692,75.20,81,96,13.11,light rain
21,21,Atuona,PF,-9.8000,-139.0333,78.15,70,0,16.26,clear sky
24,24,Puerto Ayora,EC,-0.7393,-90.3518,76.96,87,24,5.01,few clouds


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace('', np.nan)
preferred_cities_df = preferred_cities_df.dropna()

In [31]:
preferred_cities_df.count()

City_ID                264
City                   264
Country                264
Lat                    264
Lng                    264
Max Temp               264
Humidity               264
Cloudiness             264
Wind Speed             264
Weather Description    264
dtype: int64

In [45]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Lat", "Lng", "Max Temp", "Weather Description", ]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Lat,Lng,Max Temp,Weather Description,Hotel Name
2,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,broken clouds,
6,Matongo,TZ,-2.3833,34.0833,75.83,overcast clouds,
13,San Andres,CO,12.5847,-81.7006,87.82,broken clouds,
14,Vaini,TO,-21.2000,-175.2000,73.56,broken clouds,
16,Uong Bi,VN,21.0333,106.7833,71.58,overcast clouds,
17,Lodwar,KE,3.1191,35.5973,88.00,few clouds,
18,Saint George,US,37.1041,-113.5841,73.87,clear sky,
19,Rikitea,PF,-23.1203,-134.9692,75.20,light rain,
21,Atuona,PF,-9.8000,-139.0333,78.15,clear sky,
24,Puerto Ayora,EC,-0.7393,-90.3518,76.96,few clouds,


In [46]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [47]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Weather Description,Hotel Name
2,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,broken clouds,Pousada Porto Praia
6,Matongo,TZ,-2.3833,34.0833,75.83,overcast clouds,
13,San Andres,CO,12.5847,-81.7006,87.82,broken clouds,Hotel Arena Blanca
14,Vaini,TO,-21.2000,-175.2000,73.56,broken clouds,Keleti Beach Resort
16,Uong Bi,VN,21.0333,106.7833,71.58,overcast clouds,khach san Trung lien
...,...,...,...,...,...,...,...
692,Manta,EC,-0.9500,-80.7333,85.78,scattered clouds,Balandra Hotel
694,Takoradi,GH,4.8845,-1.7554,79.18,overcast clouds,Raybow International Hotel
695,Maragogi,BR,-9.0122,-35.2225,80.15,overcast clouds,Salinas do Maragogi All Inclusive Resort
696,Kilindoni,TZ,-7.9139,39.6668,79.52,clear sky,Bustani B&B


In [48]:
#hotel_df.count() # is counting empty strings

In [49]:
empty_string = (hotel_df["Hotel Name"]=="")
hotel_df["Hotel Name"].loc[empty_string].count()

20

In [50]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df

,City,Country,Lat,Lng,Max Temp,Weather Description,Hotel Name
2,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,broken clouds,Pousada Porto Praia
6,Matongo,TZ,-2.3833,34.0833,75.83,overcast clouds,NaN
13,San Andres,CO,12.5847,-81.7006,87.82,broken clouds,Hotel Arena Blanca
14,Vaini,TO,-21.2000,-175.2000,73.56,broken clouds,Keleti Beach Resort
16,Uong Bi,VN,21.0333,106.7833,71.58,overcast clouds,khach san Trung lien
...,...,...,...,...,...,...,...
692,Manta,EC,-0.9500,-80.7333,85.78,scattered clouds,Balandra Hotel
694,Takoradi,GH,4.8845,-1.7554,79.18,overcast clouds,Raybow International Hotel
695,Maragogi,BR,-9.0122,-35.2225,80.15,overcast clouds,Salinas do Maragogi All Inclusive Resort
696,Kilindoni,TZ,-7.9139,39.6668,79.52,clear sky,Bustani B&B


In [51]:
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Lat,Lng,Max Temp,Weather Description,Hotel Name
2,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,broken clouds,Pousada Porto Praia
13,San Andres,CO,12.5847,-81.7006,87.82,broken clouds,Hotel Arena Blanca
14,Vaini,TO,-21.2000,-175.2000,73.56,broken clouds,Keleti Beach Resort
16,Uong Bi,VN,21.0333,106.7833,71.58,overcast clouds,khach san Trung lien
17,Lodwar,KE,3.1191,35.5973,88.00,few clouds,Solmar Gracious Hotel
...,...,...,...,...,...,...,...
692,Manta,EC,-0.9500,-80.7333,85.78,scattered clouds,Balandra Hotel
694,Takoradi,GH,4.8845,-1.7554,79.18,overcast clouds,Raybow International Hotel
695,Maragogi,BR,-9.0122,-35.2225,80.15,overcast clouds,Salinas do Maragogi All Inclusive Resort
696,Kilindoni,TZ,-7.9139,39.6668,79.52,clear sky,Bustani B&B


In [52]:
empty_string = (hotel_df["Hotel Name"]=="")
hotel_df["Hotel Name"].loc[empty_string].count()

0

In [53]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Weather Description,Hotel Name
2,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,broken clouds,Pousada Porto Praia
13,San Andres,CO,12.5847,-81.7006,87.82,broken clouds,Hotel Arena Blanca
14,Vaini,TO,-21.2000,-175.2000,73.56,broken clouds,Keleti Beach Resort
16,Uong Bi,VN,21.0333,106.7833,71.58,overcast clouds,khach san Trung lien
17,Lodwar,KE,3.1191,35.5973,88.00,few clouds,Solmar Gracious Hotel
...,...,...,...,...,...,...,...
692,Manta,EC,-0.9500,-80.7333,85.78,scattered clouds,Balandra Hotel
694,Takoradi,GH,4.8845,-1.7554,79.18,overcast clouds,Raybow International Hotel
695,Maragogi,BR,-9.0122,-35.2225,80.15,overcast clouds,Salinas do Maragogi All Inclusive Resort
696,Kilindoni,TZ,-7.9139,39.6668,79.52,clear sky,Bustani B&B


In [54]:
# 8a. Create the output File (CSV)
output_csv_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_csv_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, country, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))